![Egeria Logo](https://raw.githubusercontent.com/odpi/egeria/master/assets/img/ODPi_Egeria_Logo_color.png)

### ODPi Egeria Hands-On Lab
# Welcome to the Conformance Test Suite Lab

## Introduction

ODPi Egeria is an open source project that provides open standards and implementation libraries to connect tools, catalogs and platforms together so they can share information (called metadata) about data and the technology that supports it.

In this hands-on lab you will get a chance to work with the conformance test suite that is used to validate that a technology can successfully join an open metadata repository cohort.

## About the Conformance Suite 

The Conformance Suite can be used to test a Platform or Repository Connector to record which Conformance 
Profiles it supports. The Conformance Suite has different Workbenches that are used to test different types of 
system. 

Initially our focus will be on the Repository Conformance Workbench. This workbench is used to test that an 
OMRS Repository Connector record which of the Repository Conformance Profiles it supports.

There are 13 repository conformance profiles in this workbench. One of them is mandatory - i.e. any repository
connector must fully support that profie in order to be certified as conformant. The other profiles are optional and 
for each of these optional profiles, a repository connector can be certified as compliant even if it does not 
provide the function required by that profile - so long as it responds appropriately to requests.


## Configuring and running the Conformance Suite 

We'll come back to the profiles later, but for now let's configure and run the Conformance Suite.

We're going to need a pair of OMAG Servers - one to run the repository under test, the other to run the workbench. The servers need to join the same cohort.

![CTS-Cohort.png](../images/CTS-Cohort.png)
> **Figure 1:** Cohort for conformance testing

When the one runnng the workbench sees the cohort registration of the server under test, it runs the workbench tests against that server's repository.

In [1]:
# STOLEN FROM COMMON/GLOBALS.IPYNB
# This file contains the common functions and definitions used in all Egeria Hands on
# Lab Notebooks. Typically these will be system-wide functions and common environment settings
#

# Disable certificate checking for local development use with self-signed certificate 
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import os
os.environ['CURL_CA_BUNDLE'] = ''

## Starting up the Egeria platforms

We'll start one OMAG Server Platform on which to run both the servers.
We also need Apache Zookeeper and Apache Kafka.  


In [31]:
#%run ../common/globals.ipynb

import requests
import pprint
import json
import os
import time

ctsPlatformURL = os.environ.get('ctsPlatformURL','https://localhost:8082') 

def checkServerPlatform(testPlatformName, testPlatformURL):
    response = requests.get(testPlatformURL + "/open-metadata/platform-services/users/garygeeke/server-platform/origin/")
    if response.status_code == 200:
        print("   ...", testPlatformName, "at", testPlatformURL, "is active - ready to begin")
    else:
        print("   ...", testPlatformName, "at", testPlatformURL, "is down - start it before proceeding")

print ("\nChecking OMAG Server Platform availability...")
checkServerPlatform("CTS OMAG Server Platform", ctsPlatformURL)

print ("Done.")


Checking OMAG Server Platform availability...
   ... CTS OMAG Server Platform at https://localhost:8082 is active - ready to begin
Done.


## Configuring the Servers

We're going to configure both the servers in the diagram above.

It's useful to create some generally useful definitions here.

Knowing both server names up front will be handy for when we configure the workbench.

To configure the servers we'll need a common cohort name and event bus configuration. 
We can let the CTS server default to using a local in-memory repository.
The CTS server does not need to run any Access Services.

In [18]:
ctsServerName    = "CTS_Server"
sutServerName    = "SUT_Server"
devCohort        = "devCohort"

We'll need to pass a couple of JSON request bodies - so let's set up a reusable header:

In [19]:
jsonContentHeader = {'content-type':'application/json'}

We'll need a JSON request body for configuration of the event bus.

In [20]:
eventBusURLroot   = os.environ.get('eventBusURLroot', 'localhost:9092')

eventBusBody      = {
    "producer": {
        "bootstrap.servers": eventBusURLroot
    },
    "consumer":{
        "bootstrap.servers": eventBusURLroot
    }
}

We'll also need a JSON request body for configuration of the workbench. 
This can be used to set the pageSize used in searches.

In [21]:
workbenchConfigBody = {
    "class"                  : "RepositoryConformanceWorkbenchConfig",
    "tutRepositoryServerName": sutServerName ,
    "maxSearchResults"       : 10
}

We also need a userId for the configuration commands. You could change this to a name you choose.

In [22]:
adminUserId      = "garygeeke"

We can perform configuration operations through the administrative interface provided by the ctsPlatformURL.

The URLs for the configuration REST APIs have a common structure and begin with the following root:

In [23]:
adminPlatformURL = ctsPlatformURL

adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'

What follows are descriptions and coded requests to configure each server.  There are a lot of common steps 
involved in configuring a metadata server, so we first define some simple 
functions that can be re-used in later steps for configuring each server.

Each function returns True or False to indicate whether it was successful.

In [24]:
def postAndPrintResult(url, json=None, headers=None):
    print("   ...... (POST", url, ")")
    response = requests.post(url, json=json, headers=headers)
    if response.status_code == 200:
        print("   ...... Success. Response: ", response.json())
        return True
    else:
        print("   ...... Failed. Response: ", response.json())
        return False
    
def getAndPrintResult(url, json=None, headers=None):
    print("   ...... (GET", url, ")")
    response = requests.get(url, json=json, headers=headers)
    if response.status_code == 200:
        print("   ...... Success. Response: ", response.json())
        return True
    else:
        print("   ...... Failed. Response: ", response.json())
        return False

def getResult(url, json=None, headers=None):
    print("\n   ...... (GET", url, ")")
    try:
        response = requests.get(url, json=json, headers=headers)
        if response.status_code == 200:
            if response.json()['relatedHTTPCode'] == 200:
                return response.json()
        return None
    except requests.exceptions.RequestException as e:
        print ("   ...... FAILED - http request threw an exception: ", e)
        return None    

def configurePlatformURL(serverName, serverPlatform):
    print("\n   ... Configuring the platform the server will run on...")
    url = adminCommandURLRoot + serverName + '/server-url-root?url=' + serverPlatform
    return postAndPrintResult(url)

def configureServerType(serverName, serverType):
    print ("\n   ... Configuring the server's type...")
    url = adminCommandURLRoot + serverName + '/server-type?typeName=' + serverType
    return postAndPrintResult(url)

def configureUserId(serverName, userId):
    print ("\n   ... Configuring the server's userId...")
    url = adminCommandURLRoot + serverName + '/server-user-id?id=' + userId
    return postAndPrintResult(url)

def configurePassword(serverName, password):
    print ("\n   ... Configuring the server's password (optional)...")
    url = adminCommandURLRoot + serverName + '/server-user-password?password=' + password
    return postAndPrintResult(url)

def configureMetadataRepository(serverName, repositoryType):
    print ("\n   ... Configuring the metadata repository...")
    url = adminCommandURLRoot + serverName + '/local-repository/mode/' + repositoryType
    return postAndPrintResult(url)

def configureDescriptiveName(serverName, collectionName):
    print ("\n   ... Configuring the short descriptive name of the metadata stored in this server...")
    url = adminCommandURLRoot + serverName + '/local-repository/metadata-collection-name/' + collectionName
    return postAndPrintResult(url)

def configureEventBus(serverName, busBody):
    print ("\n   ... Configuring the event bus for this server...")
    url = adminCommandURLRoot + serverName + '/event-bus'
    return postAndPrintResult(url, json=busBody, headers=jsonContentHeader)

def configureCohortMembership(serverName, cohortName):
    print ("\n   ... Configuring the membership of the cohort...")
    url = adminCommandURLRoot + serverName + '/cohorts/' + cohortName
    return postAndPrintResult(url)
    
def configureRepositoryWorkbench(serverName, workbenchBody):
    print ("\n   ... Configuring the repository workbench for this server...")
    url = adminCommandURLRoot + serverName + '/conformance-suite-workbenches/repository-workbench/repositories'
    return postAndPrintResult(url, json=workbenchBody, headers=jsonContentHeader)


## Configuring the CTS Server

We're going to the CTS Server from the diagram above. The CTS Server is the one that runs the repository workbench.

The server will default to using a local in-memory repository.
The CTS server does not need to run any Access Services.

Notice that when We configure the CTS Server to run the repository workbench, we provide the name of the server under test.

First we introduce a 'success' variable which is used to monitor progress in the subsequent cells.

In [25]:
success = True

In [26]:
ctsServerType          = "Conformance Suite Server"
ctsServerUserId        = "CTS1npa"
ctsServerPassword      = "CTS1passw0rd"
ctsServerPlatform      = ctsPlatformURL


print("Configuring " + ctsServerName + "...")

if (success):
    success = configurePlatformURL(ctsServerName, ctsServerPlatform)
if (success):
    success = configureServerType(ctsServerName, ctsServerType)
if (success):
    success = configureUserId(ctsServerName, ctsServerUserId)
if (success):
    success = configurePassword(ctsServerName, ctsServerPassword)
if (success):
    success = configureEventBus(ctsServerName, eventBusBody)
if (success):
    success = configureCohortMembership(ctsServerName, devCohort)
if (success):
    success = configureRepositoryWorkbench(ctsServerName, workbenchConfigBody)

if (success):
    print("\nDone.")
else:
    print("\nFAILED: please check the messages above and correct before proceeding")

Configuring CTS_Server...

   ... Configuring the platform the server will run on...
   ...... (POST https://localhost:8082/open-metadata/admin-services/users/garygeeke/servers/CTS_Server/server-url-root?url=https://localhost:8082 )
   ...... Success. Response:  {'class': 'VoidResponse', 'relatedHTTPCode': 200}

   ... Configuring the server's type...
   ...... (POST https://localhost:8082/open-metadata/admin-services/users/garygeeke/servers/CTS_Server/server-type?typeName=Conformance Suite Server )
   ...... Success. Response:  {'class': 'VoidResponse', 'relatedHTTPCode': 200}

   ... Configuring the server's userId...
   ...... (POST https://localhost:8082/open-metadata/admin-services/users/garygeeke/servers/CTS_Server/server-user-id?id=CTS1npa )
   ...... Success. Response:  {'class': 'VoidResponse', 'relatedHTTPCode': 200}

   ... Configuring the server's password (optional)...
   ...... (POST https://localhost:8082/open-metadata/admin-services/users/garygeeke/servers/CTS_Server/se

 ## Configuring the SUT Server (Server Under Test)

We're going to the SUT Server from the diagram above. The SUT Server is the one that hosts the repository that is being tested. The SUT Server will run on the same platform as the CTS Server.

The server will default to using a local in-memory repository.
The CTS server does not need to run any Access Services.

Notice that when we configure the CTS Server to run the repository workbench, we provide the name of the server under test.

In [27]:
sutServerType                  = "Metadata Repository Server"
sutServerUserId                = "SUTnpa"
sutServerPassword              = "SUTpassw0rd"
metadataCollectionName         = "SUT_MDR"
metadataRepositoryTypeInMemory = "in-memory-repository"
metadataRepositoryTypeGraph    = "local-graph-repository"

print("Configuring " + sutServerName + "...")

if (success):
    success = configurePlatformURL(sutServerName, ctsServerPlatform)
if (success):
    success = configureServerType(sutServerName, sutServerType)
if (success):
    success = configureUserId(sutServerName, sutServerUserId)
if (success):
    success = configurePassword(sutServerName, sutServerPassword)
if (success):
    success = configureMetadataRepository(sutServerName, metadataRepositoryTypeInMemory)
if (success):
    success = configureDescriptiveName(sutServerName, metadataCollectionName)
if (success):
    success = configureEventBus(sutServerName, eventBusBody)
if (success):
    success = configureCohortMembership(sutServerName, devCohort)

if (success):
    print("\nDone.")
else:
    print("\nFAILED: please check the messages above and correct before proceeding")

Configuring SUT_Server...

   ... Configuring the platform the server will run on...
   ...... (POST https://localhost:8082/open-metadata/admin-services/users/garygeeke/servers/SUT_Server/server-url-root?url=https://localhost:8082 )
   ...... Success. Response:  {'class': 'VoidResponse', 'relatedHTTPCode': 200}

   ... Configuring the server's type...
   ...... (POST https://localhost:8082/open-metadata/admin-services/users/garygeeke/servers/SUT_Server/server-type?typeName=Metadata Repository Server )
   ...... Success. Response:  {'class': 'VoidResponse', 'relatedHTTPCode': 200}

   ... Configuring the server's userId...
   ...... (POST https://localhost:8082/open-metadata/admin-services/users/garygeeke/servers/SUT_Server/server-user-id?id=SUTnpa )
   ...... Success. Response:  {'class': 'VoidResponse', 'relatedHTTPCode': 200}

   ... Configuring the server's password (optional)...
   ...... (POST https://localhost:8082/open-metadata/admin-services/users/garygeeke/servers/SUT_Server/s

The commands below deploy the server configuration documents to the server platforms where the
servers will run.

In [28]:
def deployServerToPlatform(serverName, platformURL):
    print("   ... deploying", serverName, "to the", platformURL, "platform...")
    url = adminCommandURLRoot + serverName + '/configuration/deploy'
    platformTarget = {
        "class": "URLRequestBody",
        "urlRoot": platformURL
    }
    try:
        return postAndPrintResult(url, json=platformTarget, headers=jsonContentHeader)
    except requests.exceptions.RequestException as e:
        print ("   ...... FAILED - http request threw an exception: ", e)
        return False    


print("\nDeploying server configuration documents to appropriate platforms...")
    
if (success):
    success = deployServerToPlatform(ctsServerName, ctsPlatformURL)
if (success):
    success = deployServerToPlatform(sutServerName, ctsPlatformURL)

if (success):
    print("\nDone.")
else:
    print("\nFAILED: please check the messages above and correct before proceeding")


Deploying server configuration documents to appropriate platforms...
   ... deploying CTS_Server to the https://localhost:8082 platform...
   ...... (POST https://localhost:8082/open-metadata/admin-services/users/garygeeke/servers/CTS_Server/configuration/deploy )
   ...... Success. Response:  {'class': 'VoidResponse', 'relatedHTTPCode': 200}
   ... deploying SUT_Server to the https://localhost:8082 platform...
   ...... (POST https://localhost:8082/open-metadata/admin-services/users/garygeeke/servers/SUT_Server/configuration/deploy )
   ...... Success. Response:  {'class': 'VoidResponse', 'relatedHTTPCode': 200}

Done.


## Starting the servers

We'll need to define the URL for the OMRS operational services API.

In [29]:
operationalServicesURLcore = "/open-metadata/admin-services/users/" + adminUserId

Start the CTS Server, followed by the SUT Server.

When the CTS Server sees the cohort registration for the SUT Server it will start to run the workbench.

In [45]:
def startServer(serverName, platformURL):
    print("   ... starting server", serverName, "...")
    url = platformURL + operationalServicesURLcore + '/servers/' + serverName + '/instance'
    return postAndPrintResult(url)

print ("\nStarting the CTS server ...")

if (success):
    success = startServer(ctsServerName, ctsPlatformURL)
    
# Pause to allow server to initialize fully    
time.sleep(4)

print ("\nStarting the SUT server ...")

if (success):
    success = startServer(sutServerName, ctsPlatformURL)

if (success):
    print("\nDone.")
else:
    print("\nFAILED: please check the messages above and correct before proceeding")


Starting the CTS server ...
   ... starting server CTS_Server ...
   ...... (POST https://localhost:8082/open-metadata/admin-services/users/garygeeke/servers/CTS_Server/instance )
   ...... Success. Response:  {'class': 'SuccessMessageResponse', 'relatedHTTPCode': 200, 'successMessage': 'Wed Oct 21 17:57:21 BST 2020 CTS_Server is running the following services: [Open Metadata Repository Services (OMRS), Connected Asset Services, Conformance Suite Services]'}

Starting the SUT server ...
   ... starting server SUT_Server ...
   ...... (POST https://localhost:8082/open-metadata/admin-services/users/garygeeke/servers/SUT_Server/instance )


ConnectionError: HTTPSConnectionPool(host='localhost', port=8082): Max retries exceeded with url: /open-metadata/admin-services/users/garygeeke/servers/SUT_Server/instance (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x1245a3690>: Failed to establish a new connection: [Errno 61] Connection refused'))

## Workbench Progress

The repository workbench runs a lot of tests (several thousand) and it can take a while to complete. 
There is no 'completion event' because when the confirmance suite has completed the synchronous workbench tests it 
continues to run and will perform asynchronous tests in responses to events that may be received within the cohort.
The consequence of this is that it is not easy to know when the CTS has 'finished'. However, if you scan the output
console logging from the conformance suite it is possible to detect the log output:

Thu Nov 21 09:11:01 GMT 2019 CTS_Server Information CONFORMANCE-SUITE-0011 The Open Metadata Conformance Workbench repository-workbench has completed its synchronous tests, further test cases may be triggered from incoming events.

When this has been seen you will probably see a number of further events being processed by the CTS Server. There can be up to several hundred events - that look like the following:

Thu Nov 21 09:11:03 GMT 2019 CTS_Server Event OMRS-AUDIT-8006 Processing incoming event of type DeletedEntityEvent for instance 2fd6cd97-35dd-41d9-ad2f-4d25af30033e from: OMRSEventOriginator{metadataCollectionId='f076a951-fcd0-483b-a06e-d0c7abb61b84', serverName='SUT_Server', serverType='Metadata Repository Server', organizationName='null'}

Thu Nov 21 09:11:03 GMT 2019 CTS_Server Event OMRS-AUDIT-8006 Processing incoming event of type PurgedEntityEvent for instance 2fd6cd97-35dd-41d9-ad2f-4d25af30033e from: OMRSEventOriginator{metadataCollectionId='f076a951-fcd0-483b-a06e-d0c7abb61b84', serverName='SUT_Server', serverType='Metadata Repository Server', organizationName='null'}

These events are usually DELETE and PURGE events relating to instances that have been cleaned up on the SUT Server. 

Once these events have been logged the console should go quiet. When you see this, it is possible to retrieve the workbench results from the CTS Server.

## Polling for Status
The following cell can be used to find out whether the workbench has completed its synchronous tests....

In [33]:
import pandas
from pandas.io.json import json_normalize

conformanceSuiteServicesURLcore = "/open-metadata/conformance-suite/users/" + adminUserId

def retrieveStatus(serverName, platformURL):
    print("   ... retrieving completion status from server", serverName, "...")
    url = platformURL + '/servers/' + serverName + conformanceSuiteServicesURLcore + '/status/workbenches/repository-workbench'
    return getResult(url)

print ("\nRetrieve repository-workbench status ...")

status_json = retrieveStatus(ctsServerName, ctsPlatformURL)

if (status_json != None):
    workbenchStatus = json_normalize(data = status_json['workbenchStatus'])
    workbenchId = workbenchStatus.loc[0 , "workbenchId"]
    workbenchComplete = workbenchStatus.loc[0 , "workbenchComplete"]
    if (workbenchComplete == True):
        print("\nWorkbench",workbenchId,"is complete.")
    else:
        print("\nWorkbench",workbenchId,"has not yet completed.")
else:
    print("\nFAILED: please check the messages above and correct before proceeding")


Retrieve repository-workbench status ...
   ... retrieving completion status from server CTS_Server ...

   ...... (GET https://localhost:8082/servers/CTS_Server/open-metadata/conformance-suite/users/garygeeke/status/workbenches/repository-workbench )

Workbench repository-workbench is complete.


## Retrieving the Workbench Results

The repository workbench keeps the results of the testcases in memory. When the workbench is complete (see above) you 
can request a report of the results from the REST API on the CTS Server.

The REST API has several options that supports different styles of report, but here we will request
the whole report. Note that the report is LARGE, so if you are running the Jupyter notebook server with its default configuration, the report will exceed the default max data rate for the notebook server. If you are not running the Egeria team's containers (docker/k8s), and you have not done so already, please restart the notebook server with the following configuration option:


jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10 

The returned report is also large. If the following call results in a Java Heap error you may need to increase the memory configured for your container environment, or available locally. Min 4GB, ideally 6GB additional ram is recommended for CTS.


In [36]:
report_json = None


conformanceSuiteServicesURLcore = "/open-metadata/conformance-suite/users/" + adminUserId

def retrieveReport(serverName, platformURL):
    print("   ... retrieving test report from server", serverName, "...")
    url = platformURL + '/servers/' + serverName + conformanceSuiteServicesURLcore + '/report'
    return getResult(url)

print ("\nRetrieve Conformance Suite results ...")

report_json = retrieveReport(ctsServerName, ctsPlatformURL)

if (report_json != None):
    print("\nDone.")
else:
    print("\nFAILED: please check the messages above and correct before proceeding")


Retrieve Conformance Suite results ...
   ... retrieving test report from server CTS_Server ...

   ...... (GET https://localhost:8082/servers/CTS_Server/open-metadata/conformance-suite/users/garygeeke/report )

Done.


## Making sense of the Workbench Results

The repository workbench is a large JSON object that contains different sections of interest.

The following approach uses pandas. If you have not already done so, use pip (or pip3) to install pandas.


In [37]:
import pandas
from pandas.io.json import json_normalize

if (report_json != None):
    testLabResults = json_normalize(data = report_json['testLabResults'])    
else:
    print("\nPlease check the messages above and correct before proceeding")                 

## Monitoring Progress
It's possible to ask for the results while the tests are still running - and this can be used to give an idea of progress

In [38]:
partial_result = retrieveReport(ctsServerName, ctsPlatformURL)
if (partial_result != None):
    testLabResults = json_normalize(data = report_json['testLabResults'])    
    testSkippedCount = testLabResults.loc[0 , "testSkippedCount"]
    testCaseCount = testLabResults.loc[0 , "testCaseCount"]
    testCompleteCount = testCaseCount - testSkippedCount
    print("Progress... completed ",testCompleteCount," out of a possible ",testCaseCount, " tests." )

   ... retrieving test report from server CTS_Server ...

   ...... (GET https://localhost:8082/servers/CTS_Server/open-metadata/conformance-suite/users/garygeeke/report )
Progress... completed  1703  out of a possible  1703  tests.


## Test Statistics Summary

The following is a very brief summary of the test statistics:

In [39]:
if (report_json != None):
    testRunDate = testLabResults.loc[0 , "testRunDate"]
    testCaseCount = testLabResults.loc[0 , "testCaseCount"]
    testFailedCount = testLabResults.loc[0 , "testFailedCount"]
    testSkippedCount = testLabResults.loc[0 , "testSkippedCount"]
    print("Date of conformance test:          {}".format(testRunDate))
    print("Number of tests that were run:     {}".format(testCaseCount))
    print("Number of tests that failed:       {}".format(testFailedCount))
    print("Number of tests that were skipped: {}".format(testSkippedCount))
else:
    print("\nPlease check the earlier cells and correct before proceeding")   

Date of conformance test:          2020-10-21T15:35:32.657+00:00
Number of tests that were run:     1703
Number of tests that failed:       4
Number of tests that were skipped: 0


## Conformance Profile Results

The following is a summary of the status of each conformance profile. To ensure that you get a complete summary, make sure you retrieve the results (as above) once the workbench has completed.

In [40]:
if (report_json != None):
    repositoryWorkbenchResults = json_normalize(data = report_json['testLabResults'],
                                                record_path =['testResultsFromWorkbenches','profileResults'])
    repositoryWorkbenchResultsSummary = repositoryWorkbenchResults[['id','name','description','profilePriority','conformanceStatus']]              

    display(repositoryWorkbenchResultsSummary.head(15))

,id,name,description,profilePriority,conformanceStatus
0,0,Metadata sharing,The technology under test is able to share met...,MANDATORY_PROFILE,NOT_CONFORMANT
1,1,Reference copies,"The technology under test is able to save, loc...",OPTIONAL_PROFILE,UNKNOWN_STATUS
2,2,Metadata maintenance,The technology under test supports requests to...,OPTIONAL_PROFILE,UNKNOWN_STATUS
3,3,Dynamic types,The technology under test supports changes to ...,OPTIONAL_PROFILE,UNKNOWN_STATUS
4,4,Graph queries,The technology under test supports graph-like ...,OPTIONAL_PROFILE,UNKNOWN_STATUS
5,5,Historical search,The technology under test supports search for ...,OPTIONAL_PROFILE,UNKNOWN_STATUS
6,6,Entity proxies,The technology under test is able to store stu...,OPTIONAL_PROFILE,UNKNOWN_STATUS
7,7,Soft-delete and restore,The technology under test allows an instance t...,OPTIONAL_PROFILE,UNKNOWN_STATUS
8,8,Undo an update,The technology under test is able to restore a...,OPTIONAL_PROFILE,UNKNOWN_STATUS
9,9,Reidentify instance,The technology under test supports the command...,OPTIONAL_PROFILE,UNKNOWN_STATUS
